In [1]:
import copy

import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F

from loader.DataLoader import read_dataframe
from loader.DataTransformer import lag_list, transform_matrix

from model.RandomForestLstmModel import RandomForestLstmModel
from model.ScalingResidualLstmModel import ScalingResidualLstmModel

In [2]:
import matplotlib.pyplot as plt

In [3]:
# data parameter
LAG = 15
WAVE = 3
REPEAT = 100

### Prepare Data

In [4]:
def get_file_path(wave, filename):
    curr_dir = os.getcwd()
    project_dir = curr_dir.split('GitHub')[0]
    analysis_on_covid_dir = os.path.join(project_dir, 'GitHub', 'analysis-on-covid')
    return analysis_on_covid_dir + 'checkpoint_wave' + str(wave) + '/' + filename
    # return 'checkpoint_wave' + str(wave) + '/' + filename

In [5]:
def hypothesize(dataframe):
    # taframe.loc[280:505, 'people8'] = 1
    return dataframe

In [6]:
# prepare data
sequence = read_dataframe('all')
sequence = hypothesize(sequence)
sequence = sequence.to_numpy()
y_var = np.var(sequence[:, -1])
shifted_sequence = lag_list(sequence, LAG + 1)  # shift into delayed sequences

x_train = shifted_sequence[:, :-1, 1:]  # for each delayed sequence, take all elements except last element
y_train = shifted_sequence[:, -1, -1]  # for each delayed sequence, only take the last element
y_train = y_train.reshape(-1, 1)

x_train = torch.from_numpy(x_train.astype('float64')).type(torch.Tensor)  # convert to tensor
y_train = torch.from_numpy(y_train.astype('int32')).type(torch.Tensor)  # convert to tensor

if WAVE == 1:
    x_train, y_train = x_train[52 - LAG:103 - LAG], y_train[52 - LAG:103 - LAG]
if WAVE == 2:
    x_train, y_train = x_train[160 - LAG:280 - LAG], y_train[160 - LAG:280 - LAG]
elif WAVE == 3:
    x_train, y_train = x_train[280 - LAG:505 - LAG], y_train[280 - LAG:505 - LAG]
elif WAVE == 4:
    x_train, y_train = x_train[757 - LAG:871 - LAG], y_train[757 - LAG:871 - LAG]

### Random Forest Model

In [7]:
performance = np.load(get_file_path(WAVE, 'best_train_loss.npy'))

# build model
input_dim = x_train.shape[-1]
hidden_dim = 64
num_layers = 4
output_dim = 1

models = []
for i in range(100):
    model = ScalingResidualLstmModel(input_dim, hidden_dim, num_layers, output_dim, LAG)
    model_path: str = get_file_path(WAVE, 'checkpoints/model_{:02d}.chk'.format(i))
    model.load_state_dict(torch.load(model_path))
    models.append(copy.deepcopy(model))

model = RandomForestLstmModel(models, performance)

In [8]:
# for the first day's input, the prediction is real.  
# that is to kickstart the ongoing prediction

x_test = x_train[0].reshape(1,LAG, x_train.shape[2]) # get the first input
y_pred = [ x_test[0][-1][0].detach().numpy().reshape(-1) ] # initialize the list with the first actual count

prediction_range = len(x_train)
# prediction_range = 7

for i in range(1, prediction_range):
    _ = model(x_test)
    
    # extract the next input item
    x_test = x_train[i].reshape(1, LAG, x_train.shape[2])
    
    # modify the previous day's count to be the prediction
    x_test[0][-1][0] = torch.tensor(_.item())
    
    # append to model prediction list
    _2 = _.detach().numpy()  # revert from tensor
    _2 = _2.reshape(-1)  # reshape back to normal list
    y_pred.append(_2)

In [9]:
y_pred = np.array(y_pred).reshape(-1)  # reshape back to normal list
y_train = np.array(y_train[:prediction_range]).reshape(-1)  # reshape back to normal list

In [10]:
mse = sum((y_train - y_pred) ** 2) / len(y_pred)
rmse = mse / np.var(y_train)
print("TEST | MSE: %.2E | RRSE: %.2E" % (mse, rmse))

TEST | MSE: 1.24E+02 | RRSE: 1.31E-01


In [11]:
# adjust for accumulated error
adjustment_factor = 1 - rmse * rmse
print(adjustment_factor)
y_pred *= adjustment_factor

0.9828882518614068


### Plot Result

In [12]:
# plt.plot(range(len(y_train)), y_train, color='black')
# plt.plot(range(len(y_pred)), y_pred, 'b--', label='lstm (rmse={rmse:.2e})'.format(rmse=rmse))
# # plt.plot(range(len(y_pred)), y_pred, 'b--', label='lstm')
# plt.ylabel("count")
# plt.xlabel("day")
# plt.legend()
# plt.show()